## Informational content of participant generated data

We want to assess whether the data generated by participants in the different conditions of experiment 2 and 3 are inhenrently less informative. 

Because of the confound of experiment 2, experiment 3 should be more relevant.

We will compute information gained of the data for each trial for a normative and local computation model with constant attention. 

If difficult condition lead to less information generated on average, then it constitutes evidence for the fact that different priors negatively biases the active learning process.

### Data format

Information gained will be stored as follows:


Normative OR Local Computation pd.DataFrame: 
   - index: participant_id 
   - columns: experiment, congruent, incongruent, implausible
    

### Requirements

Create a function:
   - Loop once through the trial
   - Fit/Train multiple internal states at once with different parameters sets


Input:
   1. Trial data:
      
      1.1. Dictionary - keys: id, difficulty, labels, model, judgement, data, interventions 

   2. Agents (internal_states, sensory_states, action_states)
      
      2.1. Internal states: parameter dictionary - keys: model_name, values: parameters


Output:
   - the fitted/trained internal states statistics ready for analysis

### Handling parameters

In general only the ground truth parameters for external states and the prior parameter for internal states depend on the trial, the rest should be fixed.

Therefore, it would be good to have dedicated methods to add them after the initialisation such that all models can be initialised without these. It is the case with judgement data already

#### General parameters (common to all objects):
   - Number of datapoints: $N$
   - Number of variables: $K$

#### Ornstein-Uhlenbeck network parameters:
   - Rigidity: $\theta$
   - Variance: $\sigma^2$
   - time step: $dt$

#### External states parameters
   - Ground truth model: $\mathbf{T}$

#### Internal states parameters

Note: usually, the parameters that will be fit is the temperature parameter. Maybe the decay rate for attention based models.

1. General

   - Empirical priors: $p(\Lambda)$ - can be continuous (parameters of distribution) or discrete (vector of probabilities): pass a temperature parameter as argument, then if prior judgement, then generate a prior distribution, else, generate a uniform
   - Link values: $L$
   - Temperature parameter for softmax smoothing: $\beta$
<br><br>

2. Attention based:

   - Decay rate $g$
   - Decay type: string being either "exponential" or "sigmoid$
<br><br>

3. Functional forms

   3.1. Normative and Local computations (same functional form as the OU network)
      - Rigidity: $\theta$
      - Variance: $\sigma^2$
      - time step: $dt$

   3.2 Change based IS
      - Prop constant: $c$ - if not fitting params then $c = \theta dt$
      - Samples variance: $s^2$ - major free parameter
      - Hypothesis: string representing the hypothesis about the functional form of the causal relationship
<br><br>

#### Sensory states parameters
   - Change memory parameter: $\alpha$
   - Change type: string defining how to represent change (normalised, relative, raw)

#### Action states parameters

Note: for most of the modelling exercise, all parameters are fixed. 

1. General

   - behaviour: string representing whether the agent acts or not (obs, random, actor) 
   - possible actions: vector representing the value of possible action the agent can take
   - threshold for action stopping: $\epsilon_a$ (only here for generative models to reduce unnecessary computation when convergence is achieved)
   - Policy functions: list of function relating to policies for action selection
<br><br>

2. Actions selection types

   2.1. Tree search functions

      - Action length: the length of each action (e.g. if 1, one action will be done for each frame)
      - Number of sampled models over which to do the tree search: $C$ 
      - Knowledge: False - determines whether to tree search for a given model (np.ndArray), sampling uniformly $C$ models from space ('random), sampling $C$ models from space using the posterior (False)

   2.1.1. Soft horizon
      
      - discount: $\gamma$ - how much to discount information gains
      - horizon: $\epsilon_h$ - threshold of low information at which to prune tree branch
   
   2.1.2. Hard horizon
      
      - depth: $d$ - how deep to look in the tree

   2.2. Experience Value - Acting - Obsering (vao)
      - Timestep: $dt$ - similar usage to action length, rescale action length to frames 
      - Prior parameters of actions distribution: $(\boldsymbol{\mu}, \mathbf{S})$ - Parameters for a 3D Gaussian with mean vector $\boldsymbol{\mu}$ and covariance matrix $\mathbf{S}$
      - Learning rate: $\alpha_a$ - interpolation constant defining how much the mean $\boldsymbol{\mu}$ should be updated after performing an action
      - Experience measure: string telling how experience is integrated ('change' or 'information')
      - Maximum acting time: $A_a$ maximum acting time allowed for actions (purpose is to reduce sample space to avoid underflows)
      - Maximum observing time: $O_a$ maximum observing time allowed for actions (purpose is to reduce sample space to avoid underflows)


<br><br>

In [2]:
import numpy as np
import pandas as pd
from methods.import_states import import_states_asdict
from methods.empirical_priors import generate_discrete_empirical_priors

states_dict = import_states_asdict()

for k, v in states_dict.items():
    print(k, ':', [vk for vk in v.keys()])

internal : ['normative', 'LC_discrete', 'LC_continuous', 'LC_discrete_attention', 'change_discrete', 'change_continuous']
sensory : ['omniscient']
actions : ['tree_search_soft_horizon', 'tree_search_hard_horizon', 'experience_vao']


In [3]:
g = np.array([[0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 1, 0, 0]])
g = np.array([-1/2, 1/2, -1/2, -1, 1, 0])
print(g)

[-0.5  0.5 -0.5 -1.   1.   0. ]


In [4]:
beta = 0.18
L = np.array([-1, -1/2, 0, 1/2, 1])
g_s, g_entropy = generate_discrete_empirical_priors(g, L, beta)

print(np.round(g_s, 2))
print(np.round(g_entropy, 2))

[[0.   0.99 0.   0.   0.  ]
 [0.   0.   0.   0.99 0.  ]
 [0.   0.99 0.   0.   0.  ]
 [1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.  ]
 [0.   0.   0.99 0.   0.  ]]
[0.07 0.07 0.07 0.04 0.04 0.07]


In [5]:
from scipy.optimize import minimize

def my_func(x):
    return (x - 3)**2 - 10

x0 = 50
yz = (-1, -1)
minimize(my_func, x0)

      fun: -9.999999999999973
 hess_inv: array([[0.49999999]])
      jac: array([3.57627869e-07])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 4
     njev: 6
   status: 0
  success: True
        x: array([3.00000016])

In [6]:
def foo(a, b, c, d, e):
    return a + b * (c / d) + e

a = 10
b = 2
args = [-1, 2, 1/3]
kwargs = {}
print(foo(a, b, *args, *kwargs))
print(foo(a, b, -1, 2, 1/3))

9.333333333333334
9.333333333333334


In [7]:
def build_space(K, links):
        a = links 
        c = len(links)
        s = K**2 - K

        S = np.zeros((c**s, s))

        for i in range(s):
            ou = np.tile(a, (int(c**(s-i-1)), 1)).flatten('F')
            os = tuple(ou for _ in range(c**i))
            o = np.concatenate(os)

            S[:, i] = o.T
        
        return S

def models_to_links(K, links, indexed_space, models_probs):
        s = K**2 - K
        num_links = links.size
        links_probs = np.zeros((s, num_links))
        for j in range(s):
            for k in range(num_links):
                links_probs[j, k] = np.sum(models_probs[indexed_space[:,j] == k])
            
        return links_probs

links = np.array([-1, 1])
links_index = np.array([0, 1])
K = 3

space = build_space(K, links)
indexed_space = build_space(K, links_index)

m = np.random.rand(space.shape[0])
m = m / m.sum()

In [8]:


print(m)

print(indexed_space.shape)
l = models_to_links(K, links_index, indexed_space, m)

print(l)

model = np.array([-1, 1, -1, 1, 1, -1])

model_idx = np.where((space == model).all(axis=1))[0]
print(m[model_idx])

[0.015252   0.02383223 0.01263134 0.01688933 0.00808497 0.00408933
 0.00991606 0.0169141  0.001583   0.01581441 0.02293623 0.01417448
 0.00283393 0.02321016 0.02712248 0.01963753 0.02608656 0.02220049
 0.00861806 0.00601513 0.0160108  0.01054667 0.00144193 0.02776365
 0.03352949 0.01613649 0.00143335 0.01470861 0.00100703 0.01409658
 0.02953155 0.0116397  0.01586527 0.01286985 0.01554126 0.02111502
 0.00948534 0.03287859 0.02035576 0.01735152 0.0034863  0.01665473
 0.01049026 0.02746149 0.00238106 0.03020939 0.0139851  0.00986283
 0.02123057 0.03096103 0.01665781 0.02424336 0.01103035 0.02600477
 0.01613647 0.0277127  0.02491013 0.00551112 0.01011866 0.00194567
 0.01541981 0.00377606 0.00795396 0.0207061 ]
(64, 6)
[[0.47568768 0.52431232]
 [0.49491532 0.50508468]
 [0.54573231 0.45426769]
 [0.51090374 0.48909626]
 [0.49698851 0.50301149]
 [0.43306687 0.56693313]]
[0.00144193]


In [9]:
links_stacked = np.tile(links, (model.size, 1)).T
np.where(links_stacked == model)
print(model)
print(links_stacked)

l[(links_stacked == model).T].prod()




[-1  1 -1  1  1 -1]
[[-1 -1 -1 -1 -1 -1]
 [ 1  1  1  1  1  1]]


0.013969889743927436

In [28]:
K = 3
L = np.array([-1, -1/2, 0, 1/2, 1]).astype(float)
space = build_space(K, L)

space.shape
i = 300 * 4 
j = 5**6
links = [f'link_{i+1}' for i in range(K**2-K)]
t_idx = [f'trial_{i+1}' for i in range(i)]

#a = np.zeros(i*space.shape[0])
#a = a.reshape((space.shape[0], i))
print(space.shape)
#print(a.shape)

#data = np.concatenate([space, a], axis=1)
#df = pd.DataFrame(columns=links + t_idx, data=data)
first_j = 0
a = L[first_j]

df2 = pd.DataFrame(columns=links, data=space)
df2['judgement_1 '] = df2[links[0]] == a
df2
#df2[df2[links[0]] == a | df2[links[1]] == a | df2[links[2]] == a | df2[links[3]] == a | df2[links[4]] == a | df2[links[5]] == a]

(15625, 6)


,link_1,link_2,link_3,link_4,link_5,link_6,judgement_1
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,True
1,-1.0,-1.0,-1.0,-1.0,-1.0,-0.5,True
2,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,True
3,-1.0,-1.0,-1.0,-1.0,-1.0,0.5,True
4,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,True
...,...,...,...,...,...,...,...
15620,1.0,1.0,1.0,1.0,1.0,-1.0,False
15621,1.0,1.0,1.0,1.0,1.0,-0.5,False
15622,1.0,1.0,1.0,1.0,1.0,0.0,False
15623,1.0,1.0,1.0,1.0,1.0,0.5,False


In [16]:

print(i, j)
print(i*j)
a = np.random.rand(i*j)
#a = np.zeros(i*j)
a = a.reshape((i, j))

np.savetxt('./data/param_fitting_outputs/test.csv', a, delimiter=',')

1200 15625
18750000
